
# **Install libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install datasets tqdm pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.3 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.8 MB/s eta 0:00:00


In [5]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.6/188.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=07a6086cd661817cb675905b3a91d065e42ffb3bf649fbe4546f45d11a2a89b0
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [3]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [4]:
# Check we have a GPU and check the memory size of the GPU
!nvidia-smi

Thu Sep  7 13:23:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Import packages**

In [5]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    AutoModelForSeq2SeqLM,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Set a seed**

In [6]:
import random
import numpy as np
import torch
import datasets

In [7]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)



```
# This is formatted as code
```

# ***C4-200M dataset***

In [8]:
pd.set_option('display.max_colwidth', None)

In [9]:
df = pd.read_csv('/content/drive/MyDrive/data/train_data.csv')
df.shape

(8036, 6)

In [10]:
df.head()

,Video Title,Genre,Comment,Error,Category,Correct form
0,"ওবায়দুল কাদের বললেন, ‘খেলা হবে’; আর রুমিন ফারহানা বললেন, ‘আসেন খেলি’ ! | Quader | Rumeen Farhana",Politics,কাদের কি খেলব কাদের তো খেলতেই পারে না,1,Grammatical,কাদের কি খেলবে কাদের তো খেলতেই পারে না
1,পুলিশের গাড়ির ওপর চড়াও বিএনপির কর্মীরা | BNP | Channel 24,Politics,এসব করে আরো কোন ঠাসা হবে,1,Spelling,এসব করে আরো কোণঠাসা হবে
2,Ayub Bachchu | Ek Akash Tara | আইয়ুব বাচ্চু | এক আকাশ তারা | Official Music Video,Entertainment,যুগ যুগ ধরে আমাদের মনে গেথে থাকবে এ গান,0,NaN,NaN
3,যে প্রেম কাহিনী কোন বাধা মানেনি | BBC Bangla,Miscellaneous,অাচছা অাপু এলাজী থাকলে টিকা নেওয়া জাবেনা,1,Spelling,আচ্ছা আপু এলার্জী থাকলে টিকা নেওয়া যাবেনা
4,তুরস্কের চেয়ে ভয়াবহ ভূমিকম্পের ঝুঁকিতে বাংলাদেশ | BBC Bangla,News,হে আল্লাহ এই জালিমদের থেকে আমাদের সন্তান সন্তদের কে আপনি হেফাজত করেন,0,NaN,NaN


In [11]:
from transformers import (
    AutoModelForSeq2SeqLM, AutoTokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [12]:
model_name = 'csebuetnlp/banglat5'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [13]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [14]:
train_df = pd.read_csv('/content/drive/MyDrive/data/train_data.csv')
test_df = pd.read_csv('/content/drive/MyDrive/data/test_data.csv')
stop_words_df = pd.read_excel('/content/drive/MyDrive/data/stopwords_bangla.xlsx')
train_df.shape, test_df.shape

((8036, 6), (2011, 6))

In [15]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [16]:
test_df['input_token_len'] = test_df['Comment'].apply(calc_token_len)

In [17]:
test_df.head()

,Video Title,Genre,Comment,Error,Category,Correct form,input_token_len
0,'নেত্রীর কথা শুনলে এখন হাজার হাজার শ্রমিক রাস্তায় নেমে যাবে' | Sheikh Sharhan Naser Tonmoy,Politics,আওয়ামী লীগের এতো লোক তাহলে কেন এত ভয় পায়,0,NaN,NaN,10
1,RedMagic 8 Pro দেখে আমি তো অবাক 😮,Miscellaneous,প্রাইস টা বললে কি হতো রে ইমন,1,Code Switching,দাম টা বললে কি হতো রে ইমন,8
2,অস্থির বাঙালি Part 35😂 osthir bengali | funny video | funny facts | facts bangla,Entertainment,এত সুন্দর হাসি ভালো লাগল,1,Spelling,এত সুন্দর হাসি ভালো লাগলো,6
3,দেশে প্রথমবারের মতো চ্যানেল 24-এর পর্দায় সংবাদ পাঠ করলেন এআই 'অপরাজিতা' | AI Presenter | Channel 24,News,সময় চলে এসেছে আপনাদেরকে বিদায় জানাবার,0,NaN,NaN,9
4,"মীনা, রাজু, মিঠুর মিমিক্রি করে তাক লাগিয়ে দিয়েছেন অথৈ | Oitijya Authoi Roy | Voice Artist | Somoy TV",Entertainment,চুল একটু বড় হলে ভালো হত না,1,Spelling,চুল একটু বড় হলে ভালো হতো না,8


In [18]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
train_df['input'] = train_df['Comment'].apply(lambda x: preprocess(x))
test_df['input'] = test_df['Comment'].apply(lambda x:preprocess(x))

In [19]:
train_df['Correct form'].fillna(train_df['input'],inplace=True)
test_df['Correct form'].fillna(train_df['input'],inplace=True)

In [20]:
train_df['output'] = train_df['Correct form']
test_df['output'] = test_df['Correct form']

In [21]:
train_df = train_df[['input','output']]

In [22]:
test_df = test_df[['input','output']]

In [23]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len)

In [24]:
test_df.head()

,input,output,input_token_len
0,আওয়ামী লীগের এতো লোক ভয় পায়,কাদের খেলব কাদের খেলতেই না,7
1,প্রাইস বললে হতো রে ইমন,দাম টা বললে কি হতো রে ইমন,6
2,সুন্দর হাসি ভালো লাগল,এত সুন্দর হাসি ভালো লাগলো,5
3,সময় এসেছে আপনাদেরকে বিদায় জানাবার,অাচছা অাপু এলাজী থাকলে টিকা জাবেনা,8
4,চুল একটু বড় ভালো না,চুল একটু বড় হলে ভালো হতো না,6


In [25]:
test_df['input_token_len'].describe()

count    2011.000000
mean        8.166087
std         4.620554
min         1.000000
25%         5.000000
50%         7.000000
75%        10.000000
max        66.000000
Name: input_token_len, dtype: float64

In [51]:
train_df['input_token_len'] = train_df['input'].apply(calc_token_len)

In [54]:
print(max(test_df['input_token_len']))
print(max(train_df['input_token_len']))

66
76


### We will use a token length of 64 since it will cover the vast majority of examples

In [26]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [27]:
test_dataset

Dataset({
    features: ['input', 'output', 'input_token_len'],
    num_rows: 2011
})

### Load the Dataset

In [28]:
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['output']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [29]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 3
attention_mask 3
labels 5
{'input_ids': [1158, 3409, 1], 'attention_mask': [1, 1, 1], 'labels': [604, 1247, 19504, 2461, 1]}


### Define Evaluator

In [33]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=090747694f4b592b200cd6bc2e2549271b6797bc30b08da87c7110e1ac12bfe8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [30]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

<ipython-input-30-76fa4d2ef6b0>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


### Train Model

In [31]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [39]:
!pip install transformers[torch]

In [41]:
!pip install accelerate -U

In [32]:
# defining training related arguments
batch_size = 16
args = Seq2SeqTrainingArguments(output_dir="/content/drive/MyDrive/weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=1,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")

In [33]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model,
                args=args,
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

In [35]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=83, training_loss=0.0, metrics={'train_runtime': 187.0184, 'train_samples_per_second': 42.969, 'train_steps_per_second': 0.444, 'total_flos': 202859365662720.0, 'train_loss': 0.0, 'epoch': 0.99})

In [36]:
trainer.save_model('bangla_gec_model')

In [38]:
!zip -r 'bangla_gec_model.zip' 'bangla_gec_model'

  adding: bangla_gec_model/ (stored 0%)
  adding: bangla_gec_model/generation_config.json (deflated 29%)
  adding: bangla_gec_model/training_args.bin (deflated 49%)
  adding: bangla_gec_model/spiece.model (deflated 60%)
  adding: bangla_gec_model/special_tokens_map.json (deflated 86%)
  adding: bangla_gec_model/config.json (deflated 48%)
  adding: bangla_gec_model/tokenizer.json (deflated 76%)
  adding: bangla_gec_model/pytorch_model.bin (deflated 53%)
  adding: bangla_gec_model/tokenizer_config.json (deflated 83%)


In [39]:
!mv bangla_gec_model.zip /content/drive/MyDrive/model

I have uploaded this model to HuggingFace Model Zoo and we can run inference using it

## Testing

In [40]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = 'bangla_gec_model'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [50]:
test_sentences = train_df.tail(5)['input'].tolist() + test_df.head(10)['input'].tolist()
for sentence in test_sentences:
  print(f"input sentence:",sentence)
  print("output sentence: ",correct_grammar(sentence,num_return_sequences=2))

input sentence: সত্যি শুন্য লাগে
output sentence:  ['শূন্য লাগে শুন্য শূন্য লাগে শুন্য শূন্য লাগে শুন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূণ্য লাগে শূণ্য একেবারে শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য লাগে', 'শূন্য লাগে শুন্য শূন্য লাগে শুন্য শূন্য লাগে শুন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূন্য শূন্য লাগে শূণ্য লাগে শূণ্য একেবারে শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য শূন্য লাগে শূণ্য']
input sentence: বিজ্ঞাপন গুলো চোখে পানি আসলো ভাই
output sentence:  ['ঃ বিজ্ঞাপনের বিজ্ঞাপন আই বিজ্ঞাপন আই বিজ্ঞাপন আই বিজ্ঞাপন বিজ্ঞাপন গুলোও বিজ্ঞাপন গুলোও বিজ্ঞাপন গুলো বিজ্ঞাপন গুলো বিজ্ঞাপন গুলো বিজ্ঞাপন গুলো। বিজ্ঞাপন গুলো বিজ্ঞাপন গুলো বিজ্ঞাপন বি